# 参考资料

- [python登录豆瓣并获取自己看过的电影 - Johu - 博客园](https://www.cnblogs.com/johu/p/12970233.html)

# 导入相关的包

In [1]:
import requests
from pyquery import PyQuery as pq 
import re
import csv
from fake_useragent import UserAgent
import pandas as pd

# 构造url

In [2]:
df = pd.read_csv("../data/douban_contacts.csv")

In [3]:
user_id_list = df.user_id.to_list()

In [4]:
user_id_list[0]

'atreides1030'

In [5]:
book_read_url = "https://book.douban.com/people/{user}/collect?start={num}&sort=time&rating=all&filter=all&mode=grid"
#某用户已读书籍首页
book_read_start_url = "https://book.douban.com/people/{user}/collect?start=0&sort=time&rating=all&filter=all&mode=grid"


In [9]:
# 函数：获得html  get_html(url)
# 参数说明：url为单个网址
# 返回值：html为网址的html数据，即网页源代码的字符串
def get_html(url):         # 添加：定义函数名
    headers = {
        'User-Agent': UserAgent().random
    }
    resp = requests.get(url,headers=headers)
    html = resp.text
    return html            # 添加：返回网址的html数据

In [10]:
html_tmp = get_html(test_url)

In [11]:
html_tmp

'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body>\r\n<center><h1>403 Forbidden</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n'

In [ ]:
# 判断一共有多少页

def get_total_page(url):
    headers = {
        'User-Agent': UserAgent().random
    }
    response = requests.get(url, headers=headers)
    html = response.text
    # 使用 PyQuery 解析 HTML
    doc = pq(html)
    # 选择 div.paginator 下的最后一个 a 标签，并获取其文本内容
    total_page = doc('.paginator a:last').text()  
    return int(total_page)

In [7]:
test_url = book_read_url.format(user=user_id_list[0],num="0")

test_url

'https://book.douban.com/people/atreides1030/collect?start=0&sort=time&rating=all&filter=all&mode=grid'

In [37]:
test_url2 = "https://book.douban.com"

# 请求网页

In [63]:
# 创建 UserAgent 对象
ua = UserAgent()

# 生成随机的 User-Agent 字符串
user_agent = ua.firefox
# 尝试加入user-agent
headers = {'User-Agent': user_agent}
resp = requests.get(test_url, headers=headers)  # 设置参数headers
resp #返回码200说明访问成功

<Response [403]>

In [22]:
user_agent

'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'

In [49]:
def get_html(url):         # 添加：定义函数名
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                AppleWebKit/537.36 (KHTML, like Gecko) \
                Chrome/119.0.0.0 Safari/537.36'}
    resp = requests.get(url,headers=headers)
    html = resp.text
    return html            # 添加：返回网址的html数据

In [26]:
test_html = get_html(test_url)

In [27]:
test_html

'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body>\r\n<center><h1>403 Forbidden</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n<!-- a padding to disable MSIE and Chrome friendly error page -->\r\n'

# 解析网页

In [16]:
# 函数：解析数据  extract_bookinfo_list(html)
# 参数说明：html为网页源代码的字符串
# 返回值：bookinfo_list为书籍的字典列表
def extract_bookinfo_list(html):         # 添加：定义函数名
    bookinfo_list = []
    doc = PyQuery(html)
    for book in doc.items('div.info'):
        book_name = book('h2 a').text() #书名   
        comments = book('div.short-note p').text() #短评
        rating_raw = book('div.short-note div span:first').attr("class") #评分
        rating = re.findall('\d',rating_raw)
        info_list = book('div.pub').text().split('/')   #书籍信息 
        publisher = info_list[-3].strip()   
        pub_time = info_list[-2]
        price = info_list[-1]
        authors = ''.join(info_list[:-3])
        date_raw = book('span.date').text()       
        date = re.findall('[0-9]+-[0-9]+-[0-9]+', date_raw)[0]

        bookinfo = {'book_name':book_name,
                    'authors':authors,
                    'publisher':publisher,
                    'pub_time':pub_time,
                    'comments':comments,
                    'rating':rating,
                    'date':date,
                    'price':price,
                    }

        bookinfo_list.append(bookinfo)

        
    return bookinfo_list               # 添加：返回书籍的字典列表

In [21]:
doc =PyQuery(test_html)
doc

for book in doc.items("div.info"):
    print(book("h2 a").text())

In [17]:
extract_bookinfo_list(test_html)

[]